In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
from matplotlib.gridspec import GridSpec
from matplotlib.lines import Line2D
import xarray as xr
import os; import time
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
import pickle
import h5py

In [ ]:
#MAIN DIRECTORIES
mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
scratchDirectory='/home/air673/koa_scratch/'
codeDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Project_Algorithms/Domain_Profiles'

In [ ]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str
    
def GetData(dataDirectory, parcelDirectory):
    dataNC = xr.open_dataset(dataDirectory, decode_timedelta=True) 
    parcelNC = xr.open_dataset(parcelDirectory, decode_timedelta=True) 
    return dataNC,parcelNC

def SubsetDataVars(dataNC):
    varList = ["thflux", "qvflux", "tsk", "cape", 
               "cin", "lcl", "lfc", "th",
               "prs", "rho", "qv", "qc",
               "qr", "qi", "qs","qg", 
               "buoyancy", "uinterp", "vinterp", "winterp",]
    
    varList += ["ptb_hadv", "ptb_vadv", "ptb_hidiff", "ptb_vidiff",
                "ptb_hturb", "ptb_vturb", "ptb_mp", "ptb_rdamp", 
                "ptb_rad", "ptb_div", "ptb_diss",]
    
    varList += ["qvb_hadv", "qvb_vadv", "qvb_hidiff", "qvb_vidiff", 
                "qvb_hturb", "qvb_vturb", "qvb_mp",]
    
    varList += ["wb_hadv", "wb_vadv", "wb_hidiff", "wb_vidiff",
                "wb_hturb", "wb_vturb", "wb_pgrad", "wb_rdamp", "wb_buoy",]

    return dataNC[varList]

[dataDirectory,parcelDirectory, res,t_res,Np_str,Nz_str] = GetDataDirectories(simulationNumber=1)
[data1,parcel1] = GetData(dataDirectory, parcelDirectory)

In [ ]:
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
path=dir2+'Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

#####

#Import StatisticalFunctions 
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
path=dir2+'Functions/'
sys.path.append(path)

import StatisticalFunctions
from StatisticalFunctions import * # import NumericalFunctions 

In [ ]:
##########################################################################################
#Plotting
plotting=False #KEEP FALSE WHEN JOB_ARRAYS IS RUNNING
plotting=True

In [ ]:
#READING BACK IN
# import pickle
# dir2 = dir + f'Project_Algorithms/Domain_Profiles/'
# input_file = dir2 + f'mean_lfc_{res}_{t_res}_{Np_str}.pkl'

# with open(input_file, 'rb') as f:
#     mean_LFC = pickle.load(f)
# print(mean_LFC)

def LoadMeanLFC():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        MeanLFC = pickle.load(f)
    return MeanLFC
MeanLFC=LoadMeanLFC()
mean_LFC=MeanLFC
print(f"Mean LFC is: {MeanLFC}\n")


def LoadAllCloudBase():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        all_cloudbase = pickle.load(f)
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
cloudbase=min_all_cloudbase
print(f"Minimum Cloudbase is: {cloudbase}\n")

In [ ]:
def averaged_profiles(profile):
    out_var = profile[(profile[:, 1] > 1)]  # gets rid of rows that have no data
    out_var = np.array([out_var[:, 0] / out_var[:, 1], out_var[:, 2]]).T  # divides the data column by the counter column
    return out_var

In [ ]:
def GetVars(var_names,type1):
    dir2=dir+'Project_Algorithms/Domain_Profiles/'
    input_file = dir2 + f'OUTPUT/{type1}_eulerian_entrainment_profiles_{res}_{t_res}_{Np_str}.h5'
    with h5py.File(input_file, 'r') as f:
        for var in var_names:
            globals()[f"{var}_{type1[0]}"] = f[f"profile_array_{var}_{type1[0]}"][:]

            globals()[f"{var}_{type1[0]}_SE"]=ProfileStandardError(f[f"profile_array_{var}_{type1[0]}"][:],f[f"profile_array_{var}_{type1[0]}_squares"][:])
            # globals()[f"{var}_{type1[0]}_SE"]=ProfileStandardDeviation(f[f"profile_array_{var}_{type1[0]}"][:],f[f"profile_array_{var}_{type1[0]}_squares"][:])

var_names=['e','d','net']
# # --- Load General ---
type1='general'
GetVars(var_names,type1)
# # --- Load Cloudy ---
type1='cloudy'
GetVars(var_names,type1)

In [ ]:
def load_entrainment_transition_profiles(dir_path=dir + 'Project_Algorithms/Domain_Profiles/'):
    input_file_general = dir_path + f'OUTPUT/general_eulerian_combined_entrainment_profiles_{res}_{t_res}_{Np_str}.h5'
    input_file_cloudy = dir_path + f'OUTPUT/cloudy_eulerian_combined_entrainment_profiles_{res}_{t_res}_{Np_str}.h5'

    with h5py.File(input_file_general, 'r') as f:
        c_to_g_E_general = np.array(f['profile_array_c_to_g_E'])
        c_to_g_D_general = np.array(f['profile_array_c_to_g_D'])
        g_to_c_E_general = np.array(f['profile_array_g_to_c_E'])
        g_to_c_D_general = np.array(f['profile_array_g_to_c_D'])

    with h5py.File(input_file_cloudy, 'r') as f:
        c_to_g_E_cloudy = np.array(f['profile_array_c_to_g_E'])
        c_to_g_D_cloudy = np.array(f['profile_array_c_to_g_D'])
        g_to_c_E_cloudy = np.array(f['profile_array_g_to_c_E'])
        g_to_c_D_cloudy = np.array(f['profile_array_g_to_c_D'])

    return {
        'c_to_g_E_general': c_to_g_E_general,
        'c_to_g_D_general': c_to_g_D_general,
        'g_to_c_E_general': g_to_c_E_general,
        'g_to_c_D_general': g_to_c_D_general,
        'c_to_g_E_cloudy': c_to_g_E_cloudy,
        'c_to_g_D_cloudy': c_to_g_D_cloudy,
        'g_to_c_E_cloudy': g_to_c_E_cloudy,
        'g_to_c_D_cloudy': g_to_c_D_cloudy,
    }

load_profiles = load_entrainment_transition_profiles()

c_to_g_E_general = load_profiles['c_to_g_E_general']
c_to_g_D_general = load_profiles['c_to_g_D_general'] #zero
g_to_c_E_general = load_profiles['g_to_c_E_general'] #zero
g_to_c_D_general = load_profiles['g_to_c_D_general'] #zero
c_to_g_E_cloudy  = load_profiles['c_to_g_E_cloudy'] #zero
c_to_g_D_cloudy  = load_profiles['c_to_g_D_cloudy'] #zero
g_to_c_E_cloudy  = load_profiles['g_to_c_E_cloudy']
g_to_c_D_cloudy  = load_profiles['g_to_c_D_cloudy']

In [ ]:
import matplotlib.pyplot as plt

def plot_entrainment_profiles(ax, factor=1, lw=1):
    var_names = ['e', 'd', 'net']
    colors = {'e': 'blue', 'd': 'red', 'net': 'k'}
    styles = {'g': '--', 'c': '-'}

    # --- Process Both Types ---
    out_e_g = averaged_profiles(e_g)
    out_d_g = averaged_profiles(d_g)
    out_net_g = averaged_profiles(net_g)

    out_e_c = averaged_profiles(e_c)
    out_d_c = averaged_profiles(d_c)
    out_net_c = averaged_profiles(net_c)
    
    for suffix in ['c', 'g']:
        for var in var_names:
            varname = f'out_{var}_{suffix}'
            profile = locals()[varname]
            profile_SE_name = f'{var}_{suffix}_SE'
            profile_SE = globals().get(profile_SE_name, None) 

            label_suffix = 'general' if suffix == 'g' else 'cloudy'
            ax.plot(
                profile[:, 0],
                profile[:, 1],
                color=colors[var],
                linestyle=styles[suffix],
                linewidth=lw,
                label=f'{var} ({label_suffix})'
            )

            # Uncomment below if you want to plot error shading and SE data exists
            if profile_SE is not None:
                hatch = '///' if suffix == 'g' else ""
                ax.fill_betweenx(
                    profile[:, 1],
                    profile[:, 0] - factor * profile_SE[:, 0],
                    profile[:, 0] + factor * profile_SE[:, 0],
                    color=colors[var],
                    alpha=0.1,
                    hatch=hatch
                )

    ax.grid(True)
    ax.axvline(0, color='k', linewidth=lw)
    ax.set_xlabel(r"($kg m^{-3} s^{-1}$)") 
    ax.set_ylabel('z (km)')

    # Apply scientific notation helper if you have one
    apply_scientific_notation([ax])

    # Cloud base and mean LFC lines
    ax.axhline(cloudbase, color='purple', linestyle='dashed', lw=1.2)
    ax.axhline(mean_LFC / 1000, color='green', linestyle='dashed', lw=1.2)

    # ax.set_title('Entrainment: General (dashed) vs Cloudy (solid)')
    ax.set_ylim(bottom=0)
    ax.legend()

In [ ]:
#RATIO PLOT

def safe_divide(numerator, denominator):
    # result=numerator/denominator
    # Convert to numpy arrays for safety
    numerator = np.array(numerator)
    denominator = np.array(denominator)
    
    # Mask denominator zeros or very small values to avoid divide-by-zero
    mask = (denominator == 0) | np.isnan(denominator) | np.isnan(numerator)
    
    # Create a result array filled with NaNs initially
    result = np.full_like(numerator, np.nan, dtype=float)
    
    # Only divide where denominator is nonzero and both numerator/denominator are finite
    valid = ~mask
    result[valid] = numerator[valid] / denominator[valid]
    
    return result

def plot_entrainment_transition_profiles_combined(ax, c_to_g_E, g_to_c_E, c_to_g_D, g_to_c_D):
    mean_c_to_g_E = averaged_profiles(c_to_g_E)
    mean_g_to_c_E = averaged_profiles(g_to_c_E)
    mean_c_to_g_D = averaged_profiles(c_to_g_D)
    mean_g_to_c_D = averaged_profiles(g_to_c_D)

    ax.plot(mean_g_to_c_E[:, 0], mean_g_to_c_E[:, 1], color='blue', label=f'General → Cloudy (Entrainment)',linestyle='solid')
    ax.plot(mean_c_to_g_E[:, 0], mean_c_to_g_E[:, 1], color='red', label=f'Cloudy → General (Entrainment)',linestyle='solid')
    ax.plot(mean_c_to_g_D[:, 0], mean_c_to_g_D[:, 1], color='blue', label=f'Cloudy → General (Detrainment)',linestyle='dashed')
    ax.plot(mean_g_to_c_D[:, 0], mean_g_to_c_D[:, 1], color='red', label=f'General → Cloudy (Detrainment)',linestyle='dashed')
    
    ax.axvline(0, color='k')
    ax.set_xlabel(r"($kg m^{-3} s^{-1}$)") 
    ax.set_ylabel('z (km)')

    apply_scientific_notation([ax])  # if you have this function defined

    ax.grid(True)
    ax.axhline(cloudbase, color='purple', linestyle='dashed', lw=1.2)
    ax.axhline(mean_LFC / 1000, color='green', linestyle='dashed', lw=1.2)

    # ax.set_title('Entrainment Transition: c_to_g (general) & g_to_c (cloudy)')
    ax.set_ylim(bottom=0)
    ax.legend()

    
# def plot_transfer_ratio(ax, e_g,d_g, c_to_g_general, e_c, d_c, g_to_c_cloudy, title):
#     # Apply averaging
#     mean_c_to_g = averaged_profiles(c_to_g_general)  # shape: (N, 2)
#     mean_g_to_c = averaged_profiles(g_to_c_cloudy)
    
#     # mean_e_g = averaged_profiles(e_g)
#     mean_e_c = averaged_profiles(e_c)
#     # mean_d_g = averaged_profiles(d_g)
#     mean_d_c = averaged_profiles(d_c); 
    
#     #fixing length mismatch
#     idx = np.where(mean_c_to_g[:, 1] == mean_d_c[0, 1])[0]
#     if idx.size > 0:
#         mean_c_to_g = mean_c_to_g[idx[0]:]
#     else:
#         print("No matching index found.")
    
#     ax.plot(safe_divide(mean_c_to_g[:, 0], mean_d_c[:, 0]), mean_c_to_g[:, -1], color='red',  label='Cloudy → General Entrainment\n / Cloudy Detrainment')
    
#     ax.plot(safe_divide(mean_g_to_c[:, 0], mean_e_c[:, 0]), mean_g_to_c[:, -1], color='blue', label='General → Cloudy Entrainment\n / Cloudy Entrainment')
    
#     ax.axvline(1, color='black', linestyle='dashed', linewidth=1)

#     ax.axhline(cloudbase, color='purple', linestyle='dashed', lw=1.2)
#     ax.axhline(mean_LFC / 1000, color='green', linestyle='dashed', lw=1.2)

#     ax.set_title(title)
#     ax.set_xlabel('Ratio')
#     ax.set_ylabel('Height (km)')

#     # Set x-axis limit with padding
#     pad_fraction = 0.1
#     ax.set_xlim(0, 1*1.1)

#     ax.legend()
#     apply_scientific_notation([ax])


def plot_transfer_ratio(ax, c_to_g_E,g_to_c_E,c_to_g_D,g_to_c_D, e_g,d_g,e_c,d_c, title, lw=1):
    # Apply averaging
    mean_c_to_g_E = averaged_profiles(c_to_g_E)  # shape: (N, 2)
    mean_g_to_c_E = averaged_profiles(g_to_c_E)
    mean_c_to_g_D = averaged_profiles(c_to_g_D)  # shape: (N, 2)
    mean_g_to_c_D = averaged_profiles(g_to_c_D)
    
    mean_e_g = averaged_profiles(e_g)
    mean_d_g = averaged_profiles(d_g)
    mean_e_c = averaged_profiles(e_c)
    mean_d_c = averaged_profiles(d_c)

    # Align everything to a common vertical coordinate (z), using mean_d_c
    z_ref = mean_d_c[:, 1]

    def align(profile, z_target):
        idx = np.where(profile[:, 1] == z_target[0])[0]
        return profile[idx[0]:] if idx.size > 0 else profile

    mean_c_to_g_E = align(mean_c_to_g_E, z_ref)
    mean_g_to_c_E = align(mean_g_to_c_E, z_ref)
    mean_c_to_g_D = align(mean_c_to_g_D, z_ref)
    mean_g_to_c_D = align(mean_g_to_c_D, z_ref)
    
    mean_e_g = align(mean_e_g, z_ref)
    mean_d_g = align(mean_d_g, z_ref)
    mean_e_c = align(mean_e_c, z_ref)
    # mean_d_c already aligned

    z = z_ref  # in km

    # Compute ratios
    def safe_divide(numerator, denominator, threshold=0):
        with np.errstate(divide='ignore', invalid='ignore'):
            return np.where(denominator > threshold, numerator / denominator, np.nan)

    ratio_1 = safe_divide(mean_g_to_c_E[:, 0], mean_e_c[:, 0])  # Blue
    ratio_2 = safe_divide(mean_g_to_c_D[:, 0], mean_d_g[:, 0])  # Red
    ratio_3 = safe_divide(mean_c_to_g_E[:, 0], mean_e_g[:, 0])  # Light Blue
    ratio_4 = safe_divide(mean_c_to_g_D[:, 0], mean_d_c[:, 0])  # Orange

    # Plot ratios
    ax.plot(ratio_1*100, z, color='blue', label='General → Cloudy / Cloudy Entrainment')
    ax.plot(ratio_2*100, z, color='deepskyblue', label='General → Cloudy / General Detrainment')
    ax.plot(ratio_3*100, z, color='red', label='Cloudy → General / General Entrainment')
    ax.plot(ratio_4*100, z, color='orangered', label='Cloudy → General / Cloudy Detrainment')

    ax.grid(True)
    ax.set_title(title)
    ax.set_xlabel('%')
    # ax.set_xlabel('Ratio')
    ax.set_ylabel('z (km)')
    # ax.set_xlim(-0.05, 1.05)  # Add 10% padding
    ax.set_xlim(-5, 105)  # Add 10% padding
    ax.set_ylim(bottom=0)
    ax.legend(loc='center right', fontsize=10.5-1)

    ax.axvline(0, color='k', linewidth=lw)
    # ax.axvline(1, color='black', linestyle='dashed', linewidth=1)
    ax.axvline(100, color='black', linestyle='dashed', linewidth=1)
    ax.axhline(cloudbase, color='purple', linestyle='dashed', lw=1.2)
    ax.axhline(mean_LFC / 1000, color='green', linestyle='dashed', lw=1.2)
    apply_scientific_notation([ax],decimals=2,scientific=True)

In [ ]:
def SavePlot(fig, res, t_res, Np_str, dir2, limit_y=False):
    # Define output directory
    subdir_name = f'{res}_{t_res}_{Np_str}'
    output_dir = os.path.join(dir2, 'Project_Algorithms', 'Domain_Profiles', 'PLOTS', 'Updraft_Entrainment', subdir_name)
    
    # Create the directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Choose file name
    if not limit_y:
        filename = f'Entrainment_Updraft_Profiles_{res}_{t_res}_{Np_str}.jpg'
    else:
        filename = f'SLICE/Entrainment_Updraft_Profiles_{res}_{t_res}_{Np_str}_SLICE.jpg'
    
    save_path = os.path.join(output_dir, filename)
    
    # Save the figure
    fig.savefig(save_path, bbox_inches='tight', dpi=300)
    print(f"Saved figure to: {save_path}")

In [ ]:
fig = plt.figure(figsize=(18, 6))
gs = gridspec.GridSpec(1, 3, figure=fig, wspace=0.3)

ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])

plot_entrainment_profiles(ax1)
plot_entrainment_transition_profiles_combined(ax2, c_to_g_E_general,g_to_c_E_cloudy, c_to_g_D_cloudy,g_to_c_D_general)

axes=[ax1,ax2]
#FIXING TICKS
MatchAxisLimits(axes,dim='x')

for axis in [ax1,ax2,ax3]:
    axis.set_ylim(0,20)

#ADDING AXVLINES
for ax in axes:
    ax.axvline(0,color='gray',linestyle='dashed',zorder=-10)


plot_transfer_ratio(ax3, c_to_g_E_general,g_to_c_E_cloudy,c_to_g_D_cloudy,g_to_c_D_general, e_g,d_g,e_c,d_c, title="") 
SavePlot(fig, res, t_res, Np_str, dir2)

In [ ]:
##################################################
#TESTING

In [ ]:
# #DOUBLE CHECKING
# w=data['winterp'].data
# qc=data['qc'].data
# qi=data['qi'].data
# qcqi=qc+qi
# mask = (w >= 0.5) & (qcqi >= 1e-6)
# profile_array_e_c[~mask] = np.nan
# out=np.nanmean(profile_array_e_c,axis=(0,2,3))
# plt.plot(out,data['zh'])
# ax=plt.gca()
# apply_scientific_notation([ax])

In [ ]:
# #thresholds
# w_thresh1=0.1
# w_thresh2=0.5
# qcqi_thresh=1e-6

In [ ]:
# def plotting(out_var,axis,label,color):
#     axis.plot(out_var[:,0],out_var[:,1],label=label,color=color)
#     axis.grid(True)
#     apply_scientific_notation([axis])

# fig, axs = plt.subplots(1, 3, figsize=(15, 10))
# # fig.suptitle()
# ax1,ax2,ax3=axs.flatten()

# for type in ["general","cloudy"]:
#     print(f'currently on type {type}')
#     dir2=dir+'Project_Algorithms/Domain_Profiles/'
#     if type == "general":
#         input_file = dir2+f'job_out/general_eulerian_entrainment_profiles_{res}_{Np_str}_5min.h5' 
#     elif type == "cloudy":
#         input_file = dir2+f'job_out/cloudy_eulerian_entrainment_profiles_{res}_{Np_str}_5min.h5' 

#     with h5py.File(input_file, 'r') as f:
#         if type == "general":
#             e = np.array(f['profile_e_g'])
#             d = np.array(f['profile_d_g'])
#             net = np.array(f['profile_net_g'])
#         elif type == "cloudy":
#             e = np.array(f['profile_e_c'])
#             d = np.array(f['profile_d_c'])
#             net = np.array(f['profile_net_c'])

#     #Uses Averaged_Profiles Function
#     vars = ['e','d','net'] 
    
#     for var in vars:
#         globals()[f"out_{var}"] = averaged_profiles(globals()[f"{var}"])
        
#     axises=[ax1,ax2,ax3] 
#     xlabels=['e','d','net'] 
#     for var,axis,xlabel in zip(vars,axises,xlabels):
#         if type=='general':
#             color='k'
#         elif type=='cloudy':
#             color='blue'

#         plotting(globals()[f"out_{var}"],axis,label=type+' ',color=color)
#         axis.set_ylabel('z (km)');axis.set_xlabel(xlabel);

#         if axis==ax1:
#             axis.legend()
#         axis.axvline(0,color='k')

#     type='cloudy'
#     #MEAN CLOUD BASE
#     dir2=dir+'Project_Algorithms/Domain_Profiles/'
#     if type == "general":
#         input_file = dir2+f'job_out/general_eulerian_profiles_{res}_{Np_str}_5min.h5' 
#     elif type == "cloudy":
#         input_file = dir2+f'job_out/cloudy_eulerian_profiles_{res}_{Np_str}_5min.h5'
    
#     with h5py.File(input_file, 'r') as f:
#         profile_qc = np.array(f['profile_qc'])
#         qcqi_profile=averaged_profiles(profile_qc)
#         where_cloudbase=np.where(qcqi_profile[:,0]>=qcqi_thresh)[0][0]
#         cloudbase=qcqi_profile[where_cloudbase,1]
#         for axis in axises:
#             axis.axhline(cloudbase,color='purple',linestyle='dashed')
        